# Tutorial: Inference with Shape-Shifting Traces

This tutorial introduces more advanced techniques for MCMC inference, techniques which are particularly useful when the number of random choices a model makes varies from execution to execution.

In [1]:
using Gen
using GenViz

In [2]:
server = VizServer(8094);

In [3]:
#############
# dirichlet #
#############
import Distributions
struct Dirichlet <: Distribution{Vector{Float64}} end

"""
    dirichlet(alpha::AbstractVector{T}) where {T<:Real}
Sample a `Vector{Float64}` from the Dirichlet distribution with parameter vector `alpha`.
"""
const dirichlet = Dirichlet()

function Gen.logpdf(::Dirichlet, x::AbstractVector{T}, alpha::AbstractVector{U}) where {T, U}
    Distributions.logpdf(Distributions.Dirichlet(alpha), x)
end

function Gen.logpdf_grad(::Dirichlet, x::AbstractVector{T}, alpha::AbstractVector{U}) where {T, U}
    (Distributions.gradlogpdf(Distributions.Dirichlet(alpha), x), nothing, nothing)
end

function Gen.random(::Dirichlet, alpha::AbstractVector{T}) where {T}
    rand(Distributions.Dirichlet(alpha))
end

(::Dirichlet)(alpha) = random(Dirichlet(), alpha)

Gen.has_output_grad(::Dirichlet) = true
Gen.has_argument_grads(::Dirichlet) = (false,)

#############
# geometric #
#############
import Distributions
struct Geometric <: Distribution{Int} end

"""
    geometric(p::Real)
Sample an `Int` from the Geometric distribution with parameter `p`.
"""
const geometric = Geometric()

function Gen.logpdf(::Geometric, x::Int, p::Real)
    Distributions.logpdf(Distributions.Geometric(p), x)
end

function Gen.logpdf_grad(::Geometric, x::Int, p::Real)
    (Distributions.gradlogpdf(Distributions.Geometric(p), x), nothing, nothing)
end

function Gen.random(::Geometric, p::Real)
    rand(Distributions.Geometric(p))
end

(::Geometric)(p) = random(Geometric(), p)

Gen.has_output_grad(::Geometric) = true
Gen.has_argument_grads(::Geometric) = (false,)

In [49]:
@gen function piecewise_constant(xs::Vector{Float64})
    n = @addr(poisson(1), :segment_count) + 1
    fracs = @addr(dirichlet([1.0 for i=1:n]), :fractions)
    segments = Vector{Float64}(undef, n)
    for i=1:n
        segments[i] = @addr(normal(0, 1), (:segments, i))
    end
    
    noise = @addr(gamma(1,1), :noise)
    
    min_x, max_x = minimum(xs), maximum(xs)
    for (i, x) in enumerate(xs)
        so_far, j = 0, 0
        while j < n && so_far <= (x - min_x) / (max_x - min_x)
            j += 1
            so_far += fracs[j]
        end
        @addr(normal(segments[j], noise), (:y, i))
    end
end;

In [39]:
xs_dense = collect(range(-5, stop=5, length=50))
ys_simple = fill(1., length(xs_dense)) .+ randn(length(xs_dense)) * 0.1
ys_complex = [Int(floor(abs(x/3))) % 2 == 0 ? 2 : 0 for x in xs_dense] .+ randn(length(xs_dense)) * 0.1;

In [40]:
function serialize_trace(tr)
    assmt = get_assmt(tr)
    Dict(
      "values" => [assmt[(:segments, i)] for i=1:(assmt[:segment_count]+1)],
      "fracs"  => assmt[:fractions],
      "n" => assmt[:segment_count]+1,
      "noise" => assmt[:noise],
      "y-coords" => [assmt[(:y, i)] for i=1:length(xs_dense)]
    )
end

serialize_trace (generic function with 2 methods)

In [41]:
viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])
for i=1:10
    putTrace!(viz, "t$(i)", serialize_trace(initialize(piecewise_constant, (xs_dense,))[1]))
end
displayInNotebook(viz)

Traces

In [42]:
function make_constraints(ys)
    constraints = DynamicAssignment()
    for i=1:length(ys)
        constraints[(:y, i)] = ys[i]
    end
    constraints
end

make_constraints (generic function with 1 method)

In [43]:
viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])
for i=1:10
    (tr, _) = importance_resampling(piecewise_constant, (xs_dense,), make_constraints(ys_simple), 3000)
    putTrace!(viz, "t$(i)", serialize_trace(tr))
end
displayInNotebook(viz)

Traces

In [45]:
viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])
scores = Vector{Float64}(undef, 10)
for i=1:10
    (tr, _) = importance_resampling(piecewise_constant, (xs_dense,), make_constraints(ys_complex), 50000)
    scores[i] = get_score(tr)
    putTrace!(viz, "t$(i)", serialize_trace(tr))
end
displayInNotebook(viz)
println(logsumexp(scores)-log(10))

Traces

-26.035256011352963


In [11]:
import Random

In [46]:
function simple_update(tr)
    (tr, _) = mh(tr, select(:segment_count, :fractions))
    (tr, _) = mh(tr, select(:fractions))
    (tr, _) = mh(tr, select(:noise))
    for i=1:(get_assmt(tr)[:segment_count]+1)
        (tr, _) = mh(tr, select((:segments, i)))
    end
    tr
end

Random.seed!(3)
viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])

sc = displayInNotebook(viz) do
    (tr, _) = initialize(piecewise_constant, (xs_dense,), make_constraints(ys_complex))
    for iter=1:500
        tr = simple_update(tr)
        putTrace!(viz, "t", serialize_trace(tr))
        sleep(0.01)
    end
    return get_score(tr)
end
println(sc)

Traces

43.07157861977393


In [47]:
for i=1:10
    (tr, _) = initialize(piecewise_constant, (xs_dense,), make_constraints(ys_complex))
    for iter=1:500
        tr = simple_update(tr)
    end
    scores[i]=get_score(tr)
end
println(logsumexp(scores)-log(10))

43.51724755504296


<hr />

### Exercise: Analyzing the algorithm's behavior
Try running the algorithm with random seed 3, then with random seed 4. In both cases, the initial trace has two segments, one long one and one short one. Why does the chain get stuck in the run with random seed 4, but not the run with random seed 3?

*Your answer here*

<hr />

In [48]:
@gen function segments_proposal(t, xs, ys)
    min_x, max_x = minimum(xs), maximum(xs)
    x_range = max_x - min_x
    n = @addr(poisson(1), :segment_count) + 1
    fracs = @addr(dirichlet([1.0 for i=1:n]), :fractions)
    segments = Vector{Float64}(undef, n)
    for i=1:n
        min = min_x + x_range * sum(fracs[1:i-1])
        max = min_x + x_range * sum(fracs[1:i])
        relevant_ys = [y for (x,y) in zip(xs,ys) if x >= min && x <= max]
        segments[i] = @addr(normal(sum(relevant_ys)/length(relevant_ys), 0.3), (:segments, i))
    end
end

DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Any, Any, Any], getfield(Main, Symbol("##107#115"))(), getfield(Main, Symbol("##111#119"))(), Bool[false, false, false], false)

In [50]:
function custom_update(tr)
    (tr, _) = mh(tr, segments_proposal, (xs_dense,ys_complex))
    (tr, _) = mh(tr, select(:fractions))
    (tr, _) = mh(tr, select(:noise))
    for i=1:(get_assmt(tr)[:segment_count]+1)
        (tr, _) = mh(tr, select((:segments, i)))
    end
    tr
end

viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])

scores = displayInNotebook(viz) do
    for i=1:10
        (tr, _) = initialize(piecewise_constant, (xs_dense,), make_constraints(ys_complex))
        for iter=1:500
            tr = custom_update(tr)
            putTrace!(viz, "t$(i)", serialize_trace(tr))
            sleep(0.001)
        end
        scores[i] = get_score(tr)
    end
    scores
end

println(logsumexp(scores)-log(10))

Traces

38.68685539598271


In [17]:
@gen function split_or_merge(t)
    choices = get_assmt(t)
    old_n = choices[:segment_count] + 1
    
    split_or_merge = @addr(bernoulli(old_n == 1 ? 1 : 0.3), :split_or_merge)
    if split_or_merge
        # split
        idx = @addr(uniform_discrete(1,old_n), :index)
        @addr(beta(1,1), :split_percentage)
        @addr(normal(choices[(:segments, idx)], 0.1), :new_value_1)
        @addr(normal(choices[(:segments, idx)], 0.1), :new_value_2)
    else
        # merge
        idx = @addr(uniform_discrete(1, old_n-1), :index) # merge i and i+1
        @addr(normal((choices[(:segments, idx)] + choices[(:segments, idx+1)]) / 2.0, 0.1), :new_value)
    end
end

function involution(t, fwd_assmt, fwd_ret, proposal_args)
    split_or_merge = fwd_assmt[:split_or_merge]
    old_choices = get_assmt(t)
    n = old_choices[:segment_count] + (split_or_merge ? 2 : 0)
    new_choices = DynamicAssignment()
    bwd_assmt = DynamicAssignment()
    old_fracs = old_choices[:fractions]
    fracs = Vector{Float64}(undef, n)
    idx = fwd_assmt[:index]
    for i=1:idx-1
        fracs[i] = old_fracs[i]
    end
    
    if split_or_merge
        # split
        new_choices[:segment_count] = old_choices[:segment_count] + 1
        proportion = fwd_assmt[:split_percentage]
        fracs[idx] = proportion * old_fracs[idx]
        fracs[idx+1] = (1 - proportion) * old_fracs[idx]
        new_choices[(:segments, idx)] = fwd_assmt[:new_value_1]
        new_choices[(:segments, idx+1)] = fwd_assmt[:new_value_2]
        bwd_assmt[:new_value] = old_choices[(:segments, idx)]
        
        for i=idx+2:n
            fracs[i] = old_fracs[i-1]
            new_choices[(:segments, i)] = old_choices[(:segments, i-1)]
        end
        
        new_choices[:fractions] = fracs
        bwd_assmt[:split_or_merge] = false
        bwd_assmt[:index] = idx
    else
        #merge
        new_choices[:segment_count] = old_choices[:segment_count] - 1
        fracs[idx] = old_fracs[idx] + old_fracs[idx+1]
        new_choices[(:segments, idx)] =  fwd_assmt[:new_value]
        for i=idx+1:n
            fracs[i] = old_fracs[i+1]
            new_choices[(:segments, i)] = old_choices[(:segments, i+1)]
        end
        new_choices[:fractions] = fracs
        bwd_assmt[:split_or_merge] = true
        bwd_assmt[:index] = idx
        bwd_assmt[:new_value_1] = old_choices[(:segments, idx)]
        bwd_assmt[:new_value_2] = old_choices[(:segments, idx+1)]
        bwd_assmt[:split_percentage] = old_fracs[idx] / fracs[idx]
    end
    (new_trace, weight) = force_update(get_args(t), noargdiff, t, new_choices)
    (new_trace, bwd_assmt, weight)
end

involution (generic function with 1 method)

In [ ]:
@gen function mean_segments_proposal(t, xs, ys, i)
    min_x, max_x = minimum(xs), maximum(xs)
    x_range = max_x - min_x
    fracs = get_assmt(t)[:fractions]
    min = min_x + x_range * sum(fracs[1:i-1])
    max = min_x + x_range * sum(fracs[1:i])
    relevant_ys = [y for (x,y) in zip(xs,ys) if x >= min && x <= max]
    @addr(normal(sum(relevant_ys)/length(relevant_ys), 0.3), (:segments, i))
end

function custom_update_inv(tr)
    (tr, _) = mh(tr, split_or_merge, (), involution)
    for i=1:(get_assmt(tr)[:segment_count]+1)
        (tr, _) = mh(tr, mean_segments_proposal, (xs_dense, ys_complex, i))
    end
    (tr, _) = mh(tr, select(:noise))
    tr
end

viz = Viz(server, joinpath(@__DIR__, "piecewise-constant-viz/dist"), [xs_dense])

scores = displayInNotebook(viz) do
    for i=1:10
        (tr, _) = initialize(piecewise_constant, (xs_dense,), make_constraints(ys_complex))
        for iter=1:3000
            tr = custom_update_inv(tr)
            putTrace!(viz, "t$(i)", serialize_trace(tr))
        end
        scores[i] = get_score(tr)
    end
    scores
end

println(logsumexp(scores)-log(10))

# Gaussian processes

In [3]:
import LinearAlgebra

"""Node in a tree representing a covariance function"""
abstract type Node end
abstract type LeafNode <: Node end
abstract type BinaryOpNode <: Node end

"""
    size(::Node)
Number of nodes in the subtree rooted at this node.
"""
Base.size(::LeafNode) = 1
Base.size(node::BinaryOpNode) = node.size

"""Constant kernel"""
struct Constant <: LeafNode
    param::Float64
end


eval_cov(node::Constant, x1, x2) = node.param

function eval_cov_mat(node::Constant, xs::Vector{Float64})
    n = length(xs)
    fill(node.param, (n, n))
end


"""Linear kernel"""
struct Linear <: LeafNode
    param::Float64
end


eval_cov(node::Linear, x1, x2) = (x1 - node.param) * (x2 - node.param)

function eval_cov_mat(node::Linear, xs::Vector{Float64})
    xs_minus_param = xs .- node.param
    xs_minus_param * xs_minus_param'
end


"""Squared exponential kernel"""
struct SquaredExponential <: LeafNode
    length_scale::Float64
end


eval_cov(node::SquaredExponential, x1, x2) =
    exp(-0.5 * (x1 - x2) * (x1 - x2) / node.length_scale)

function eval_cov_mat(node::SquaredExponential, xs::Vector{Float64})
    diff = xs .- xs'
    exp.(-0.5 .* diff .* diff ./ node.length_scale)
end


"""Periodic kernel"""
struct Periodic <: LeafNode
    scale::Float64
    period::Float64
end


function eval_cov(node::Periodic, x1, x2)
    freq = 2 * pi / node.period
    exp((-1/node.scale) * (sin(freq * abs(x1 - x2)))^2)
end


function eval_cov_mat(node::Periodic, xs::Vector{Float64})
    freq = 2 * pi / node.period
    abs_diff = abs.(xs .- xs')
    exp.((-1/node.scale) .* (sin.(freq .* abs_diff)).^2)
end


"""Plus node"""
struct Plus <: BinaryOpNode
    left::Node
    right::Node
    size::Int
end


Plus(left, right) = Plus(left, right, size(left) + size(right) + 1)


function eval_cov(node::Plus, x1, x2)
    eval_cov(node.left, x1, x2) + eval_cov(node.right, x1, x2)
end


function eval_cov_mat(node::Plus, xs::Vector{Float64})
    eval_cov_mat(node.left, xs) .+ eval_cov_mat(node.right, xs)
end


"""Times node"""
struct Times <: BinaryOpNode
    left::Node
    right::Node
    size::Int
end


Times(left, right) = Times(left, right, size(left) + size(right) + 1)


function eval_cov(node::Times, x1, x2)
    eval_cov(node.left, x1, x2) * eval_cov(node.right, x1, x2)
end


function eval_cov_mat(node::Times, xs::Vector{Float64})
    eval_cov_mat(node.left, xs) .* eval_cov_mat(node.right, xs)
end


const CONSTANT = 1      # 0.2
const LINEAR = 2        # 0.2
const SQUARED_EXP = 3   # 0.2
const PERIODIC = 4      # 0.2
const PLUS = 5          # binary 0.1
const TIMES = 6         # binary 0.1

const node_type_to_num_children = Dict(
    CONSTANT => 0,
    LINEAR => 0,
    SQUARED_EXP => 0,
    PERIODIC => 0,
    PLUS => 2,
    TIMES => 2)

const max_branch = 2
const node_dist = Float64[0.2, 0.2, 0.2, 0.2, 0.1, 0.1]

"""Compute covariance matrix by evaluating function on each pair of inputs."""
function compute_cov_matrix(covariance_fn::Node, noise, xs)
    n = length(xs)
    cov_matrix = Matrix{Float64}(undef, n, n)
    for i=1:n
        for j=1:n
            cov_matrix[i, j] = eval_cov(covariance_fn, xs[i], xs[j])
        end
        cov_matrix[i, i] += noise
    end
    return cov_matrix
end


"""Compute covariance function by recursively computing covariance matrices."""
function compute_cov_matrix_vectorized(covariance_fn, noise, xs)
    n = length(xs)
    eval_cov_mat(covariance_fn, xs) + Matrix(noise * LinearAlgebra.I, n, n)
end

function compute_predictive(covariance_fn::Node, noise::Float64,
                            xs::Vector{Float64}, ys::Vector{Float64},
                            new_xs::Vector{Float64})
    n_prev = length(xs)
    n_new = length(new_xs)
    means = zeros(n_prev + n_new)
    cov_matrix = compute_cov_matrix(covariance_fn, noise, vcat(xs, new_xs))
    cov_matrix_11 = cov_matrix[1:n_prev, 1:n_prev]
    cov_matrix_22 = cov_matrix[n_prev+1:n_prev+n_new, n_prev+1:n_prev+n_new]
    cov_matrix_12 = cov_matrix[1:n_prev, n_prev+1:n_prev+n_new]
    cov_matrix_21 = cov_matrix[n_prev+1:n_prev+n_new, 1:n_prev]
    @assert cov_matrix_12 == cov_matrix_21'
    mu1 = means[1:n_prev]
    mu2 = means[n_prev+1:n_prev+n_new]
    conditional_mu = mu2 + cov_matrix_21 * (cov_matrix_11 \ (ys - mu1))
    conditional_cov_matrix = cov_matrix_22 - cov_matrix_21 * (cov_matrix_11 \ cov_matrix_12)
    conditional_cov_matrix = 0.5 * conditional_cov_matrix + 0.5 * conditional_cov_matrix'
    (conditional_mu, conditional_cov_matrix)
end

"""
Predict output values for some new input values
"""
function predict_ys(covariance_fn::Node, noise::Float64,
                    xs::Vector{Float64}, ys::Vector{Float64},
                    new_xs::Vector{Float64})
    (conditional_mu, conditional_cov_matrix) = compute_predictive(
        covariance_fn, noise, xs, ys, new_xs)
    mvnormal(conditional_mu, conditional_cov_matrix)
end

predict_ys

# Model

In [9]:
@gen function covariance_prior()
    node_type = @addr(categorical(node_dist), :type)

    if node_type == CONSTANT
        param = @addr(uniform_continuous(0, 1), :param)
        node = Constant(param)

    # linear kernel
    elseif node_type == LINEAR
        param = @addr(uniform_continuous(0, 1), :param)
        node = Linear(param)

    # squared exponential kernel
    elseif node_type == SQUARED_EXP
        length_scale= @addr(uniform_continuous(0, 1), :length_scale)
        node = SquaredExponential(length_scale)

    # periodic kernel
    elseif node_type == PERIODIC
        scale = @addr(uniform_continuous(0, 1), :scale)
        period = @addr(uniform_continuous(0, 1), :period)
        node = Periodic(scale, period)

    # plus combinator
    elseif node_type == PLUS
        left = @addr(covariance_prior(), :left)
        right = @addr(covariance_prior(), :right)
        node = Plus(left, right)

    # times combinator
    elseif node_type == TIMES
        left = @addr(covariance_prior(), :left)
        right = @addr(covariance_prior(), :right)
        node = Times(left, right)

    # unknown node type
    else
        error("Unknown node type: $node_type")
    end

    return node
end

@gen function model(xs::Vector{Float64})
    n = length(xs)
    covariance_fn::Node = @addr(covariance_prior(), :tree)
    noise = @addr(gamma(1, 1), :noise) + 0.01
    cov_matrix = compute_cov_matrix_vectorized(covariance_fn, noise, xs)
    @addr(mvnormal(zeros(n), cov_matrix), :ys)
    return covariance_fn
end;

In [10]:
function serialize_trace(tr, xmin, xmax)
    choices = get_assmt(tr)
    (xs,) = get_args(tr)
    curveXs = collect(Float64, range(xmin, length=100, stop=xmax))
    curveYs = [predict_ys(get_retval(tr), 0.0001, xs, choices[:ys],curveXs) for i=1:5]
    Dict("y-coords" => choices[:ys],
        "curveXs" => curveXs, "curveYs" => curveYs)
end

serialize_trace (generic function with 1 method)

In [8]:
viz = Viz(server, "gp-viz/dist", [collect(Float64, -1:0.1:1)]);
for iter=1:20
    (tr, _) = initialize(model, (collect(Float64, -1:0.1:1),))
    putTrace!(viz, "t$(iter)", serialize_trace(tr, -1, 1))
end
displayInNotebook(viz)

Traces <polyline data-v-e47e7d44="" points="22.060000000000002, 164.67175988443262 23.842626262626265, 174.57446069598575 25.62525252525253, 145.6459419380351 27.407878787878786, 105.09274653687862 29.19050505050505, 90.48486856025201 30.97313131313131, 107.06300914436794 32.75575757575758, 102.24816949898931 34.538383838383844, 77.13194478793774 36.3210101010101, 87.89170124643786 38.10363636363636, 170.53740489651608 39.88626262626262, 174.09169645044264 41.66888888888889, 143.14130075106578 43.45151515151515, 173.53788252066911 45.23414141414142, 198.54 47.016767676767685, 150.02263951986131 48.79939393939394, 93.86092123071656 50.5820202020202, 87.3784795478864 52.36464646464647, 119.41411540070331 54.14727272727273, 115.47708978956003 55.929898989898994, 101.0097673477888 57.71252525252525, 92.27564391624772 59.49515151515151, 159.67089240409382 61.27777777777778, 179.3513188388681 63.06040404040404, 149.7560959514762 64.8430303030303, 162.05732777463962 66.62565656565656, 158.45770115366935 68.40828282828284, 126.3933226930189 70.19090909090909, 97.87345064163614 71.97353535353535, 81.58951676111998 73.75616161616162, 110.25487736325242 75.53878787878787, 110.08714564418992 77.32141414141415, 88.25350526496533 79.1040404040404, 73.63231292350903 80.88666666666667, 105.47118581002593 82.66929292929294, 120.68418017424837 84.45191919191919, 129.37770807558894 86.23454545454545, 138.2699262625796 88.0171717171717, 106.63979581901083 89.79979797979799, 92.92133532744583 91.58242424242424, 71.08774520868434 93.3650505050505, 82.50950148083005 95.14767676767677, 118.8765685750452 96.93030303030304, 108.01242407458774 98.71292929292929, 80.87318069835237 100.49555555555555, 51.451953045044974 102.2781818181818, 40.460550755655134 104.06080808080807, 61.4270262564319 105.84343434343434, 92.46340527912572 107.6260606060606, 78.57785087703533 109.40868686868687, 70.52030769190742 111.19131313131314, 91.35379391929642 112.97393939393939, 69.32222589356749 114.75656565656567, 98.66295299614224 116.53919191919192, 130.90018223894788 118.32181818181817, 110.97890399972695 120.10444444444444, 79.80286499800216 121.8870707070707, 47.983830733658124 123.66969696969697, 39.60248675867874 125.45232323232322, 62.01967678880749 127.23494949494949, 92.55116708444199 129.01757575757574, 38.54614823490874 130.800202020202, 22.060000000000002 132.5828282828283, 83.34550835082119 134.36545454545455, 102.05409345750053 136.1480808080808, 106.51140311866806 137.93070707070706, 114.72237599848512 139.7133333333333, 108.12567268960355 141.4959595959596, 102.34999062399734 143.27858585858587, 84.13003797759103 145.06121212121212, 93.70134034005399 146.84383838383837, 101.3976063854834 148.62646464646463, 97.14357312088204 150.4090909090909, 56.70542063773192 152.19171717171716, 23.998984045363585 153.97434343434344, 48.75475843151693 155.7569696969697, 105.53478768693714 157.53959595959597, 133.84794187320995 159.32222222222222, 121.8223615611331 161.10484848484847, 131.19717573497115 162.88747474747476, 155.88186634244963 164.670101010101, 150.5406756509043 166.45272727272726, 129.00693460205918 168.23535353535354, 104.5500234004351 170.0179797979798, 92.44297916484531 171.80060606060607, 98.54538214942389 173.58323232323232, 79.84251624671214 175.36585858585858, 58.457164925945335 177.14848484848483, 106.26202263876127 178.9311111111111, 175.96418676524183 180.71373737373736, 152.12101544422902 182.49636363636364, 148.71999904786824 184.2789898989899, 186.3479646577467 186.06161616161614, 190.70026317262864 187.84424242424242, 132.31925848440238 189.62686868686868, 85.07783896339161 191.40949494949493, 97.08568081386599 193.1921212121212, 120.85203704740258 194.97474747474746, 111.11933770690648 196.7573737373737, 93.59573810255577 198.54, 110.96606023687983" style="fill: none; stroke: rgba(0, 200, 0, 0.498039); stroke-width: 1px;"> <polyline data-v-e47e7d44="" points="22.060000000000002, 162.91932809040983 23.842626262626265, 167.62032124296445 25.

In [11]:
function initialize_trace(xs::Vector{Float64}, ys::Vector{Float64})
    constraints = DynamicAssignment()
    constraints[:ys] = ys
    (trace, _) = initialize(model, (xs,), constraints)
    return trace
end

initialize_trace (generic function with 1 method)

In [24]:
function dsl_expression(assmt::Assignment)
    node_type = assmt[:type]
    if node_type == CONSTANT
        return "$(assmt[:param])"
    elseif node_type == LINEAR
        return "LINEAR($(assmt[:param]))"
    elseif node_type == SQUARED_EXP
        return "SQUARED_EXP($(assmt[:length_scale]))"
    elseif node_type == PERIODIC
        return "PERIODIC(scale = $(assmt[:scale]), period = $(assmt[:period]))"
    elseif node_type == PLUS
        return "($(dsl_expression(get_subassmt(assmt, :left))) + $(dsl_expression(get_subassmt(assmt, :right))))"
    else
        return "($(dsl_expression(get_subassmt(assmt, :left))) * $(dsl_expression(get_subassmt(assmt, :right))))"
    end
end

dsl_expression (generic function with 1 method)

In [12]:
@gen function random_node_path(n::Node)
    if @addr(bernoulli(isa(n, LeafNode) ? 1.0 : 0.5), :stop)
        return :tree
    else
        (next_node, direction) = @addr(bernoulli(0.5), :left) ? (n.left, :left) : (n.right, :right)
        rest_of_path = @addr(random_node_path(next_node), :rest_of_path)
        if isa(rest_of_path, Pair)
            return :tree => direction => rest_of_path[2]
        else
            return :tree => direction
        end
    end
end

@gen function regen_random_subtree(prev_trace)
    @addr(covariance_prior(), :new_subtree)
    @addr(random_node_path(get_retval(prev_trace)), :path)
end

function subtree_involution(trace, fwd_assmt::Assignment, path_to_subtree, proposal_args::Tuple)
    # Need to return a new trace, a bwd_assmt, and a weight.
    model_assmt = get_assmt(trace)
    bwd_assmt = DynamicAssignment()
            
    set_subassmt!(bwd_assmt, :path, get_subassmt(fwd_assmt, :path))
    set_subassmt!(bwd_assmt, :new_subtree, get_subassmt(model_assmt, :tree))
    
    new_trace_update = DynamicAssignment()
    set_subassmt!(new_trace_update, path_to_subtree, get_subassmt(fwd_assmt, :new_subtree))
    (new_trace, weight, _, _) = force_update(get_args(trace), noargdiff, trace, new_trace_update)
    (new_trace, bwd_assmt, weight)
end


function run_mcmc(trace, iters::Int)
    for iter=1:iters
        (trace, _) = mh(trace, regen_random_subtree, (), subtree_involution)
        (trace, _) = mh(trace, select(:noise))
    end
    return trace
end

function extract_cov_noise(trace)
    cov = get_retval(trace)
    noise = get_assmt(trace)[:noise]
    return (cov, noise)
end

extract_cov_noise (generic function with 1 method)

In [13]:
#########################
# load airline data set #
#########################
import CSV
using StatsBase: mean
function get_airline_dataset()
    df = CSV.read("$(@__DIR__)/airline.csv")
    xs = df[1]
    ys = df[2]
    xs .-= minimum(xs) # set x minimum to 0.
    xs /= maximum(xs) # scale x so that maximum is at 1.
    ys .-= mean(ys) # set y mean to 0.
    ys *= 4 / (maximum(ys) - minimum(ys)) # make it fit in the window [-2, 2]
    return (xs, ys)
end

get_airline_dataset (generic function with 1 method)

In [16]:
(xs, ys) = get_airline_dataset();

In [17]:
function run_mcmc(trace, iters::Int, viz)
    for iter=1:iters
        (trace, acc) = mh(trace, regen_random_subtree, (), subtree_involution)
        (trace, _) = mh(trace, select(:noise))
        if acc
            putTrace!(viz, "t", serialize_trace(trace, -1, 2))
            sleep(0.1)
        end
    end
    return trace
end

run_mcmc (generic function with 2 methods)

In [19]:
get_retval(t)

Times(SquaredExponential(0.30395121768610545), Periodic(0.33894605990344573, 0.33402820371415465), 3)

In [18]:
viz = Viz(server, "gp-viz/dist", [xs])
t = initialize_trace(xs, ys);
putTrace!(viz, "t", serialize_trace(t, -1, 2))
t = displayInNotebook(viz) do
    sleep(1)
    run_mcmc(t, 1000, viz)
end;

Traces <polyline data-v-e47e7d44="" points="50, 307.1520915395712 54.040404040404034, 286.96529292937225 58.08080808080807, 381.03472616290816 62.121212121212125, 427.76207484912237 66.16161616161617, 421.5816345397614 70.20202020202021, 385.64041167016404 74.24242424242424, 303.02964457842535 78.28282828282829, 330.9654833232409 82.32323232323233, 439.67873713968805 86.36363636363636, 430.12247538585814 90.4040404040404, 450 94.44444444444446, 336.22097971155824 98.48484848484848, 307.6104872330592 102.52525252525251, 393.5287667058419 106.56565656565655, 430.91305704374435 110.60606060606061, 423.88038635623934 114.64646464646465, 406.3452359059269 118.68686868686868, 328.4111231013357 122.72727272727272, 340.4539488815815 126.76767676767676, 406.1980085358413 130.8080808080808, 393.63294689914227 134.84848484848484, 406.5272165958037 138.8888888888889, 337.7135792951956 142.92929292929296, 325.7680989409404 146.96969696969697, 359.2440247588735 151.010101010101, 361.6240358040111 155.05050505050502, 354.75266319010245 159.09090909090907, 359.4557142713037 163.1313131313131, 328.7953360907879 167.17171717171718, 327.7436366388831 171.21212121212122, 340.1238387707119 175.25252525252526, 335.1293874894442 179.2929292929293, 333.94825825346794 183.33333333333331, 322.7608895557627 187.37373737373736, 319.3775926438436 191.41414141414143, 323.19020051699744 195.45454545454544, 318.56532056741395 199.49494949494948, 311.62873467210653 203.53535353535355, 320.98296524581315 207.5757575757576, 311.644208431479 211.6161616161616, 307.16688156518217 215.65656565656565, 313.9589734896946 219.6969696969697, 310.70324121524305 223.73737373737373, 306.10965784205814 227.77777777777777, 307.7734982077669 231.81818181818184, 301.96408168286507 235.85858585858588, 306.23895769924195 239.8989898989899, 303.5777948803953 243.93939393939394, 290.09721925165957 247.97979797979798, 303.41581513430805 252.02020202020202, 293.1749514752942 256.06060606060606, 276.3704163314554 260.10101010101005, 296.5006709696908 264.14141414141415, 287.9557234260982 268.18181818181813, 272.39594057212 272.2222222222222, 289.2540527569223 276.2626262626262, 277.96690205508355 280.3030303030303, 272.78795321766887 284.34343434343435, 281.2555659338596 288.38383838383834, 265.15539323377 292.42424242424244, 281.047545695292 296.46464646464653, 273.3103586210616 300.5050505050505, 244.17566104055942 304.5454545454545, 273.9997695595366 308.5858585858586, 263.5312147893753 312.62626262626264, 229.77378399833265 316.66666666666663, 266.55643756864595 320.7070707070707, 267.5655639800679 324.7474747474747, 244.3943119655586 328.78787878787875, 252.06641531193864 332.82828282828285, 220.3852302085425 336.86868686868684, 226.41002549183807 340.9090909090909, 248.36167401304746 344.9494949494949, 258.39124529447315 348.98989898989896, 253.95292839554952 353.030303030303, 189.49753685192945 357.07070707070704, 150.9174808515017 361.1111111111111, 199.7442342758813 365.15151515151507, 283.79019771916774 369.1919191919192, 242.59062066301777 373.2323232323232, 191.49603595804615 377.2727272727273, 109.94219177714382 381.3131313131313, 115.35993854483587 385.35353535353534, 206.63798760176383 389.3939393939394, 294.2726086596567 393.4343434343434, 227.37410650096103 397.47474747474746, 93.99263188015766 401.51515151515156, 58.47844360022265 405.5555555555556, 111.76871661235845 409.59595959595964, 288.3877127400299 413.6363636363637, 248.38559011085528 417.67676767676767, 154.58779073287036 421.71717171717177, 50 425.75757575757575, 60.646009589862956 429.7979797979798, 178.3165331071317 433.8383838383839, 298.4938215925546 437.8787878787879, 216.0427137057426 441.9191919191919, 108.20118282012606 445.95959595959596, 79.82886604892684 450, 110.04367344588297" style="fill: none; stroke: rgba(0, 200, 0, 0.498039); stroke-width: 1px;"> <polyline data-v-e47e7d44="" points="50, 307.37339528181394 54.040404040404034, 296.27397129838073 58.08080808080807, 353.0044967977971 62.12121